<a href="https://colab.research.google.com/github/Marcin19721205/MachineLearningBootCamp/blob/main/03_feature_extraction01MJ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### scikit-learn
Strona biblioteki: [https://scikit-learn.org](https://scikit-learn.org)  

Dokumentacja/User Guide: [https://scikit-learn.org/stable/user_guide.html](https://scikit-learn.org/stable/user_guide.html)

Podstawowa biblioteka do uczenia maszynowego w języku Python.

Aby zainstalować bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install scikit-learn
```
Aby zaktualizować do najnowszej wersji bibliotekę scikit-learn, użyj polecenia poniżej:
```
!pip install --upgrade scikit-learn
```
Kurs stworzony w oparciu o wersję `0.22.1`

### Spis treści:
1. [Import bibliotek](#0)
2. [Załadowanie danych](#1)
3. [Utworzenie kopii danych](#2)
4. [Generowanie nowych zmiennych](#3)
5. [Dyskretyzacja zmiennej ciągłej](#4)
6. [Ekstrakcja cech](#5)



### <a name='0'></a> Import bibliotek

In [1]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.6.1'

### <a name='1'></a> Załadowanie danych

Extrakcja danych z notowan akcji AMAZON

In [2]:
def fetch_financial_data(company='AMZN'):
    """
    This function fetches stock market quotations.
    """
    import pandas_datareader.data as web
    return web.DataReader(name=company, data_source='stooq')

df_raw = fetch_financial_data()
df_raw.head()

,Open,High,Low,Close,Volume
Date,,,,,
2025-10-17,214.560,214.80,211.0300,213.04,45986944
2025-10-16,215.670,218.59,212.8101,214.47,42414591
2025-10-15,216.620,217.71,212.6600,215.57,45909469
2025-10-14,215.555,219.32,212.6000,216.39,45665580
2025-10-13,217.700,220.68,217.0400,220.07,37809650


In [13]:
df.index.month_name


<bound method _inherit_from_data.<locals>.method of DatetimeIndex(['2025-10-17', '2025-10-16', '2025-10-15', '2025-10-14',
               '2025-10-13'],
              dtype='datetime64[ns]', name='Date', freq=None)>

### <a name='2'></a> Utworzenie kopii danych

Volume była w int64, konwertyujemy ją na float64

In [7]:
df = df_raw.copy()
df['Volume'] = df['Volume'].astype('float64')
df = df[:5]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2025-10-17 to 2025-10-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      float64
dtypes: float64(5)
memory usage: 240.0 bytes


### <a name='3'></a> Generowanie nowych zmiennych

In [ ]:
df.index.month

Int64Index([1, 1, 1, 1, 1], dtype='int64', name='Date')

Układanie nowych kolumn z daty - day, month, year. to musimy zrobić do ML

In [16]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2025-10-17,214.560,214.80,211.0300,213.04,45986944.0,17,10,2025
2025-10-16,215.670,218.59,212.8101,214.47,42414591.0,16,10,2025
2025-10-15,216.620,217.71,212.6600,215.57,45909469.0,15,10,2025
2025-10-14,215.555,219.32,212.6000,216.39,45665580.0,14,10,2025
2025-10-13,217.700,220.68,217.0400,220.07,37809650.0,13,10,2025


### <a name='4'></a> Dyskretyzacja zmiennej ciągłej

In [17]:
df = pd.DataFrame(data={'height': [175., 178.5, 185., 191., 184.5, 183., 168.]})
df

,height
0,175.0
1,178.5
2,185.0
3,191.0
4,184.5
5,183.0
6,168.0


Grupowanie zmiennych ciągłych na kategorie, dobra taki histogram  
# Segmentacja

In [27]:
df['height_cat'] = pd.cut(x=df.height, bins=3) # sam dopasuj podział na 3 kategorie
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


In [28]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195)) #dostajemy 3 przedziały 160->175, 175->180, 180->195
df

,height,height_cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


zamiast przedziału, wyświetlamy konkretną etykietę

In [31]:
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195), labels=['small', 'medium', 'high'])
df

,height,height_cat
0,175.0,small
1,178.5,medium
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


Get dummies, wyodrebnia nam segmentacje do nowych kolumn

pamietamy, o drop first, która jest nadmiarowa i trzeba ją usunac

In [32]:
pd.get_dummies(df, drop_first=True, prefix='height')

,height,height_medium,height_high
0,175.0,False,False
1,178.5,True,False
2,185.0,False,True
3,191.0,False,True
4,184.5,False,True
5,183.0,False,True
6,168.0,False,False


### <a name='5'></a> Ekstrakcja cech

In [33]:
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


Wywietli nam długość listy

In [35]:
df['lang_number'] = df['lang'].apply(len)
df

,lang,lang_number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


deficja funkcji lambda (daj flage=1 jesli PL, jesli nie ma PL to 0)

wyszukuje w df w wierszu, czy mamy wartość PL

In [37]:
df['PL_flag'] = df['lang'].apply(lambda x: 1 if 'PL' in x else 0)
df

,lang,lang_number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


generujemy df jako kolumne website z adresem strony

In [40]:
df = pd.DataFrame(data={'website': ['wp.pl', 'onet.pl', 'google.com','dupa.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com
3,dupa.com


Podziel tekst względem '.' na poszczególne kolumny

In [41]:
df.website.str.split('.', expand=True)

,0,1
0,wp,pl
1,onet,pl
2,google,com
3,dupa,com


In [42]:
new = df.website.str.split('.', expand=True)
df['portal'] = new[0]
df['extension'] = new[1]
df

,website,portal,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
3,dupa.com,dupa,com
